## The purpose of this notebook is to test the preliminary LDA model trained from HydroShare public data to see if the model can be used for HydroShare keyword recommendation

In [1]:
# install the needed modules
!pip install gensim
!pip install nltk
!pip install pyLDAvis
!pip install ipython_blocking

In [2]:
# import the needed modules
import re
from gensim.models import Phrases
from gensim.models.ldamodel import LdaModel
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from collections import defaultdict
from nltk import pos_tag
import ipywidgets as widgets
from IPython.display import display
from hs_restclient import HydroShare, HydroShareAuthBasic
import pyLDAvis.gensim
import ipython_blocking
import warnings
warnings.filterwarnings('ignore')

In [3]:
lemmatizer = WordNetLemmatizer()

# add stop words including custom stop words to remove from the input text
stop_words = stopwords.words('english')
custom_stop_words = ['include', 'included', 'includes', 'including', 'contain', 'contains', 'containing', 'file', 'et', 'al', 'different', 
                    'dat', 'edu', 'not', 'would', 'say', 'could', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 
                     'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'from', 'published', 
                     'run', 'need', 'even', 'right', 'line', 'also', 'may', 'take', 'come', 'used', 'using', 'use', 'public', 
                     'private', 'new', 'west', 'east', 'south', 'north', 'approximately', 'four', 'low', 'high', 'available', 'mile', 'daily', 
                     'year', 'month', 'variable', 'square', 'city', 'foot', 'area', 'within', 'provide', 'firm', 'file', 'state', 'information', 
                     'well', 'university']
stop_words.extend(custom_stop_words)

In [4]:
# load the trained LDA model
model_path = "model/hs_lda_model"
lda = LdaModel.load(model_path)

In [5]:
def process_text(text):
    # Make all the strings lowercase
    text = text.lower()
    # remove urls
    text = re.sub('http[s]?://\S+', '', text)
    # remove non alphabetic characters except dash symbol which is converted to underbar that indicates phrases
    text = re.sub('[-]', '_', text)
    text = re.sub('[^A-Za-z_]', ' ', text.lower())

    # Tokenize the text; this is, separate every sentence into a list of words
    # Since the text is already split into sentences you don't have to call sent_tokenize
    tokenized_text = word_tokenize(text)
    
    tag_map = defaultdict(lambda : wn.NOUN)    
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV    
    
    # Remove the stopwords and lemmatize each word
    clean_text = [
        lemmatizer.lemmatize(word[1:] if word.startswith('_') else word, tag_map[tag[0]]) for word, tag in pos_tag(tokenized_text)
        if word not in stop_words and len(word)>2
    ]
    # this final output is a list of words - remove stop words from lemmatized text to completely remove stop words
    return [word for word in clean_text if word not in stop_words]

### Input any abstract text for testing the model prediction of keywords. Note that the model is trained with HydroShare public data, so the scope of keyword prediction is limited to the content covered by the training data, and as more HydroShare public data become available, the model will be updated regularly with new data incorporated in training.

In [6]:
abstract_input = widgets.Textarea(
    placeholder="Input the abstract to extract keywords from", 
    disabled=False)
display(abstract_input)
#print(dir(abstract_input))
def abstract_input_callback(widget):
    global abstract
    abstract = abstract_input.value
    return
abstract_input.on_trait_change(abstract_input_callback)
button = widgets.Button(description="Continue to run the rest of cells below")
display(button)

<IPython.core.display.Javascript object>

In [9]:
%blockrun button

In [10]:
# process the input text
print("Input abstract is:", abstract)
doc = process_text(abstract)
# Add bigrams and trigrams to texts
# higher threshold means few phrases, heavily depends on scoring function, which can be set as 'default' or 'npmi'
bigram = Phrases(doc, min_count=1, threshold=1, scoring='default') 
trigram = Phrases(bigram[doc])

for token in bigram[doc]:
    if '_' in token and token not in doc:
        # Token is a bigram, add to document.
        doc.append(token)
        # print("in bigram processing: ", token)
for token in trigram[doc]:
    if '_' in token and token not in doc:
        # Token is a bigram, add to document.
        doc.append(token)
        # print("in trigram processing: ", token)

Input abstract is: The data set contains the monthly statistics for the APCPsfc variable (precipitation total) of the North American Land Data Assimilation System Version 2 (NLDAS-2) model. The period of analysis is from 1979-01-02 to 2013-12-31. The statistics for each calendar month are the mean, standard deviation, minimum, maximum, and percentiles in 0.05 interval. The data set also includes a p-value per calendar month of the Kolmogorov-Smirnov (KS) test. The p-value of the KS test shows if the computed empirical cumulative distribution function (CDF) comes from a fitted gamma distribution


### Use the model to predict topics of the input abstract text, only output the most likely topic and its keyword representation

In [11]:
# use the model to predict topics of the input abstract text, only output the most likely topic and its keyword representation
test_bows = [lda.id2word.doc2bow(doc)]
for i, row_list in enumerate(lda[test_bows]):
    row = row_list[0] if lda.per_word_topics else row_list
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0:
            wp = lda.show_topic(topic_num)
            topic_keywords_list = [word for word, prop in wp]
            topic_keywords = "; ".join(topic_keywords_list)
            print("propbability of topic: ", prop_topic)
            print(topic_keywords)            
        else:
            break

propbability of topic:  0.987118766366
model; data; test; input; simulation; climate; output; landlab; resource; water


### Update the model by inrementally incorporating the new data and then recommend keywords

In [12]:
# update the model by inrementally incorporating the new data and then recommend keywords
lda.update(test_bows)
for i, row_list in enumerate(lda[test_bows]):
    row = row_list[0] if lda.per_word_topics else row_list
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0:
            wp = lda.show_topic(topic_num)
            topic_keywords_list = [word for word, prop in wp]
            topic_keywords = "; ".join(topic_keywords_list)
            print("After model update")
            print("propbability of topic: ", prop_topic)
            print(topic_keywords)            
        else:
            break

After model update
propbability of topic:  0.987954944261
data; model; test; set; distribution; statistic; calendar; p_value; analysis; period


### Explore the model with original training data visually

In [13]:
import pickle
with open("model/training_corpus.dat", "rb") as fp:
    corpus = pickle.load(fp)

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary=lda.id2word, sort_topics=False)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0       4.333509        1       1 -0.006067  0.099699
1       4.280527        1       2 -0.121876  0.101845
2      11.726443        1       3  0.005445 -0.077589
3       6.721783        1       4  0.254366 -0.117027
4      18.805228        1       5  0.126835  0.081870
5      10.558076        1       6  0.037384  0.173035
6       3.479859        1       7 -0.175598 -0.190689
7       2.363813        1       8 -0.198997 -0.148246
8       4.363243        1       9  0.054080 -0.094574
9       4.439442        1      10 -0.009377  0.022423
10      9.325517        1      11  0.118572  0.032020
11     11.558454        1      12  0.160927 -0.108229
12      2.027228        1      13 -0.186364  0.033916
13      6.016879        1      14 -0.059331  0.191546, topic_info=      Category         Freq                Term        Total  loglift  logprob
term                                                                          
2157   Default  1270.000000                soil  1270.000000  30.0000  30.0000
3775   Default  4927.000000                data  4927.000000  29.0000  29.0000
17509  Default  1842.000000               model  1842.000000  28.0000  28.0000
11253  Default   985.000000       precipitation   985.000000  27.0000  27.0000
14823  Default  2060.000000               river  2060.000000  26.0000  26.0000
17516  Default   573.000000            historic   573.000000  25.0000  25.0000
2018   Default  1899.000000           watershed  1899.000000  24.0000  24.0000
13049  Default   827.000000               usace   827.000000  23.0000  23.0000
7902   Default   685.000000           discharge   685.000000  22.0000  22.0000
7554   Default   719.000000                 cwm   719.000000  21.0000  21.0000
11847  Default  1092.000000         temperature  1092.000000  20.0000  20.0000
6249   Default   374.000000            republic   374.000000  19.0000  19.0000
267    Default   373.000000           dominican   373.000000  18.0000  18.0000
10102  Default   679.000000               creek   679.000000  17.0000  17.0000
18111  Default   401.000000              pequea   401.000000  16.0000  16.0000
497    Default  2758.000000               water  2758.000000  15.0000  15.0000
7370   Default   701.000000                test   701.000000  14.0000  14.0000
3371   Default   561.000000        distribution   561.000000  13.0000  13.0000
2623   Default   474.000000             p_value   474.000000  12.0000  12.0000
9031   Default   450.000000                 mmw   450.000000  11.0000  11.0000
3600   Default   404.000000            dissolve   404.000000  10.0000  10.0000
8223   Default   497.000000           statistic   497.000000   9.0000   9.0000
11336  Default   507.000000            calendar   507.000000   8.0000   8.0000
11750  Default   666.000000          hydroshare   666.000000   7.0000   7.0000
18645  Default   576.000000              sample   576.000000   6.0000   6.0000
16219  Default   738.000000                 set   738.000000   5.0000   5.0000
12243  Default   373.000000  model_my_watershed   373.000000   4.0000   4.0000
4148   Default   838.000000                land   838.000000   3.0000   3.0000
14884  Default   931.000000                site   931.000000   2.0000   2.0000
15636  Default   436.000000             support   436.000000   1.0000   1.0000
...        ...          ...                 ...          ...      ...      ...
7603   Topic14    98.108057       soil_moisture   112.698082   2.6720  -5.0067
17391  Topic14    16.362962              litter    17.768998   2.7282  -6.7977
14321  Topic14    29.205579           microbial    32.517151   2.7032  -6.2184
385    Topic14    87.654914            luquillo   120.084326   2.4958  -5.1193
13237  Topic14    40.192120    soil_temperature    50.889780   2.5746  -5.8991
15384  Topic14    70.303118                 gas   101.462469   2.4437  -5.3399
4537   Topic

### Explore the model with test abstract data visually

In [14]:
vis = pyLDAvis.gensim.prepare(lda, test_bows, dictionary=lda.id2word, sort_topics=False)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0       0.074802        1       1 -0.006067  0.099699
1       0.071430        1       2 -0.121876  0.101845
2       0.154243        1       3  0.005445 -0.077589
3      98.795494        1       4  0.254366 -0.117027
4       0.137222        1       5  0.126835  0.081870
5       0.102279        1       6  0.037384  0.173035
6       0.097842        1       7 -0.175598 -0.190689
7       0.064135        1       8 -0.198997 -0.148246
8       0.070707        1       9  0.054080 -0.094574
9       0.055573        1      10 -0.009377  0.022423
10      0.122881        1      11  0.118572  0.032020
11      0.115232        1      12  0.160927 -0.108229
12      0.052344        1      13 -0.186364  0.033916
13      0.085814        1      14 -0.059331  0.191546, topic_info=      Category      Freq                      Term     Total  loglift  logprob
term                                                                          
3775   Default  2.000000                      data  2.000000  30.0000  30.0000
17509  Default  1.000000                     model  1.000000  29.0000  29.0000
11253  Default  0.000000             precipitation  0.000000  28.0000  28.0000
4148   Default  0.000000                      land  0.000000  27.0000  27.0000
8327   Default  0.000000                  analysis  0.000000  26.0000  26.0000
18115  Default  0.000000                    system  0.000000  25.0000  25.0000
2544   Default  0.000000                     total  0.000000  24.0000  24.0000
16219  Default  1.000000                       set  1.000000  23.0000  23.0000
2041   Default  0.000000                  interval  0.000000  22.0000  22.0000
7370   Default  1.000000                      test  1.000000  21.0000  21.0000
10060  Default  0.000000                     input  0.000000  20.0000  20.0000
13210  Default  0.000000                      mean  0.000000  19.0000  19.0000
12592  Default  0.000000                   version  0.000000  18.0000  18.0000
3072   Default  0.000000                      show  0.000000  17.0000  17.0000
13947  Default  0.000000                  standard  0.000000  16.0000  16.0000
3725   Default  0.000000                   climate  0.000000  15.0000  15.0000
11336  Default  1.000000                  calendar  1.000000  14.0000  14.0000
497    Default  0.000000                     water  0.000000  13.0000  13.0000
17531  Default  0.000000                  resource  0.000000  12.0000  12.0000
1716   Default  0.000000                simulation  0.000000  11.0000  11.0000
17744  Default  0.000000                    period  0.000000  10.0000  10.0000
14023  Default  0.000000                   process  0.000000   9.0000   9.0000
7811   Default  0.000000                      flow  0.000000   8.0000   8.0000
10195  Default  0.000000                    output  0.000000   7.0000   7.0000
15238  Default  0.000000                    result  0.000000   6.0000   6.0000
4418   Default  0.000000                    change  0.000000   5.0000   5.0000
15006  Default  0.000000                       per  0.000000   4.0000   4.0000
7155   Default  0.000000                streamflow  0.000000   3.0000   3.0000
2018   Default  0.000000                 watershed  0.000000   2.0000   2.0000
18647  Default  0.000000                  function  0.000000   1.0000   1.0000
...        ...       ...                       ...       ...      ...      ...
15670  Topic14  0.000054                   methane  0.000114   6.3160  -6.5664
14141  Topic14  0.000504                    sensor  0.001077   6.3018  -4.3385
3879   Topic14  0.000060    electrical_resistivity  0.000131   6.2862  -6.4643
18276  Topic14  0.000051                 cczo_soil  0.000111   6.2861  -6.6227
7782   Topic14  0.000058            carbon_dioxide  0.000125   6.2858  -6.5052
11410  Topic14  0.000545                     depth  0.001296   6.1951  -4.2599
17450  Topic

## Input your feedback to have them captured and added to the HydroShare resource

In [15]:
# login to HydroShare using your username and password
username = input("Please input your HydroShare user name followed by enter key: ")

Please input your HydroShare user name: hongyi


In [16]:
import getpass
print("Please input your password followed by enter key:")
password = getpass.getpass()

········


In [17]:
auth = HydroShareAuthBasic(username=username, password=password)
hs = HydroShare(auth=auth)

In [18]:
# input your feedback 
feedback_input = widgets.Textarea(
    placeholder="Input your feedback comment which will be captured and added to the HydroShare resource", 
    disabled=False)
display(feedback_input)

feedback_file_name = "feedback_{}.txt".format(username)
#print(dir(abstract_input))
def feedback_input_callback(widget):
    global feedback_text
    feedback_text = feedback_input.value
    return
feedback_input.on_trait_change(feedback_input_callback)
cont_button = widgets.Button(description='Continue to run the rest of cells below')
display(cont_button)

<IPython.core.display.Javascript object>

In [21]:
%blockrun cont_button

In [22]:
# write user feedback to file
f = open(feedback_file_name, "w")
f.write(feedback_text)
f.close()


In [24]:
# add feedback into feedback resource
feedback_res_id = "b0abfd43ba1949ef8b69bead62785788"
hs.addResourceFile(feedback_res_id, feedback_file_name)

{'file_name': 'feedback_hongyi.txt',
 'file_path': 'feedback_hongyi.txt',
 'resource_id': 'b0abfd43ba1949ef8b69bead62785788'}